# Credit score classification project

In [443]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk

In [444]:
train = pd.read_csv('train.csv')
#test = pd.read_csv('test.csv')

C:\Users\Aleks\AppData\Local\Temp\ipykernel_23284\3503458761.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train.csv')


In [445]:
train.head(5)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [446]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

# Data Preprocessing

In [447]:
train = train.drop(['Name', 'SSN'], axis='columns') # no added value, all the information in  Customer_ID
train = train.drop(['ID'], axis='columns') # redundant too

In [448]:
# alter datatypes where necessary
columns_to_alter = ['Age', 'Annual_Income', 'Num_of_Loan','Num_of_Delayed_Payment',
                    'Changed_Credit_Limit', 'Outstanding_Debt',
                    'Amount_invested_monthly', 'Monthly_Balance']
train[columns_to_alter] = train[columns_to_alter].apply(pd.to_numeric, errors='coerce', axis=1)

In [449]:
# encode columns where necessary (Credit_Mix, Payment_of_Min_Amount, Credit_Score)
# mapping
credit_mix_dict = {'Bad':0, 'Standard':1, 'Good':2, '_':float('nan')}
poma_dict = {'No':0, 'Yes':1, 'NM':float('nan')}
credit_score_dict = {'Poor':0, 'Standard':1, 'Good':2}

train['Payment_of_Min_Amount'] = train['Payment_of_Min_Amount'].map(poma_dict)
train['Credit_Mix'] = train['Credit_Mix'].map(credit_mix_dict)
train['Credit_Score'] = train['Credit_Score'].map(credit_score_dict)

In [450]:
# after dealing with column types we may support ourselves with EDA report
from pandas_profiling import ProfileReport
profile = ProfileReport(train.sample(frac=0.1), title="Credit Score Profiling Report", explorative=True)
profile.to_file("credit_score.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [451]:
# eliminate the most irrelevant columns
for colname in train.columns.values[:-1]:
    try:
        if train[colname].corr(train['Credit_Score']) < 0.1:
            train = train.drop(colname, axis='columns')
    except:
        pass # not with every columns the correlation could have been measured, to deal with later

In [452]:
# another report to check what columns are irrelevant
profile = ProfileReport(train.sample(frac=0.1), title="Credit Score Profiling Report", explorative=True)
profile.to_file("credit_score2.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [453]:
# drop some more less relevant columns
train = train.drop(['Month', 'Occupation', 'Payment_Behaviour'], axis='columns')
# Payment_Behaviour/Credit Score correlation is 0.083, I expected higher 

In [454]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Customer_ID              100000 non-null  object 
 1   Monthly_Inhand_Salary    84998 non-null   float64
 2   Type_of_Loan             88592 non-null   object 
 3   Credit_Mix               79805 non-null   float64
 4   Credit_History_Age       90970 non-null   object 
 5   Amount_invested_monthly  91216 non-null   float64
 6   Monthly_Balance          98791 non-null   float64
 7   Credit_Score             100000 non-null  int64  
dtypes: float64(4), int64(1), object(3)
memory usage: 6.1+ MB


In [455]:
# processing Credit_History_Age so that's continuous
import re
def history_age(age):
    try : 
        years = int(re.findall('[0-9]+', age)[0])
        month = int(re.findall('[0-9]+', age)[1])
        return years*12 + month
    except :
        return np.nan

In [456]:
train['Credit_History_Age'] = train['Credit_History_Age'].apply(history_age)

In [457]:
# filling missing values
# Monthly_Inhand_Salary, Amount_invested_monthly, Monthly_Balance  are continuous so we may
# replace the null values using 'mean' for particular customers identified by Customer_ID

continuous_values = ['Monthly_Inhand_Salary', 'Amount_invested_monthly',
                     'Monthly_Balance', 'Credit_History_Age']
[train[column].fillna(train.groupby('Customer_ID')[column].transform('mean'), inplace=True)
 for column in continuous_values]

[None, None, None, None]

In [458]:
# Credit_Mix value is discrete but we may assign to missing rows the values
# from other rows with the same Customer_ID

# using mean is not causing any trouble here, we're practically taking the value that appears
# in the rows with the same id
train['Credit_Mix'].fillna(train.groupby('Customer_ID')['Credit_Mix'].transform('mean'), inplace=True)

In [459]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Customer_ID              100000 non-null  object 
 1   Monthly_Inhand_Salary    100000 non-null  float64
 2   Type_of_Loan             88592 non-null   object 
 3   Credit_Mix               100000 non-null  float64
 4   Credit_History_Age       100000 non-null  float64
 5   Amount_invested_monthly  100000 non-null  float64
 6   Monthly_Balance          100000 non-null  float64
 7   Credit_Score             100000 non-null  int64  
dtypes: float64(5), int64(1), object(2)
memory usage: 6.1+ MB


In [460]:
# type of loan may contain some interesting information
train['Type_of_Loan']

0        Auto Loan, Credit-Builder Loan, Personal Loan,...
1        Auto Loan, Credit-Builder Loan, Personal Loan,...
2        Auto Loan, Credit-Builder Loan, Personal Loan,...
3        Auto Loan, Credit-Builder Loan, Personal Loan,...
4        Auto Loan, Credit-Builder Loan, Personal Loan,...
                               ...                        
99995                          Auto Loan, and Student Loan
99996                          Auto Loan, and Student Loan
99997                          Auto Loan, and Student Loan
99998                          Auto Loan, and Student Loan
99999                          Auto Loan, and Student Loan
Name: Type_of_Loan, Length: 100000, dtype: object

In [461]:
# first we fill the information for customers that have other rows with full information available
train['Type_of_Loan'].fillna(train.groupby('Customer_ID')['Type_of_Loan'].first(), inplace=True)
# later we fill with ''
train['Type_of_Loan'].fillna('', inplace=True)

In [462]:
types =  train['Type_of_Loan'].value_counts().head(9).index[1:] 
for loan_type in types: # the single types of loans
    train[loan_type] = train['Type_of_Loan'].str.contains(loan_type)

In [463]:
del train['Type_of_Loan']

In [464]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Customer_ID              100000 non-null  object 
 1   Monthly_Inhand_Salary    100000 non-null  float64
 2   Credit_Mix               100000 non-null  float64
 3   Credit_History_Age       100000 non-null  float64
 4   Amount_invested_monthly  100000 non-null  float64
 5   Monthly_Balance          100000 non-null  float64
 6   Credit_Score             100000 non-null  int64  
 7   Not Specified            100000 non-null  bool   
 8   Credit-Builder Loan      100000 non-null  bool   
 9   Personal Loan            100000 non-null  bool   
 10  Debt Consolidation Loan  100000 non-null  bool   
 11  Student Loan             100000 non-null  bool   
 12  Payday Loan              100000 non-null  bool   
 13  Mortgage Loan            100000 non-null  bool   
 14  Auto 

# Model

In [465]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [466]:
y = train['Credit_Score']
del train['Credit_Score']
del train['Customer_ID']

In [467]:
X_train, X_test, y_train, y_test = train_test_split(train,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=2137,
                                                    stratify=y)

# Random Forest

In [468]:
from sklearn.ensemble import RandomForestClassifier

In [469]:
rf_model = RandomForestClassifier(n_estimators=120)
rf_model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=120)

In [470]:
y_pred = rf_model.predict(X_test)

In [471]:
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.71      0.77      0.74      8097
           1       0.80      0.77      0.79     16511
           2       0.69      0.69      0.69      5392

    accuracy                           0.76     30000
   macro avg       0.74      0.74      0.74     30000
weighted avg       0.76      0.76      0.76     30000

